In [73]:
import numpy as np

import pandas as pd

import math

from sklearn.model_selection import train_test_split

from scipy import stats

from sklearn.neighbors import KNeighborsClassifier

pd.options.mode.chained_assignment = None

from IPython.display import HTML


In [74]:
f = open("data/student-mat.csv", "r")

lines = f.readlines()

for x in range(len(lines)):
    lines[x] = lines[x].replace(';',',')

f2 = open("data/student-mat-processado.csv", "w")

for x in lines:
    f2.write(x)


df = pd.read_csv("data/student-mat-processado.csv")

In [75]:
def converter_binario(coluna):
    unicos = coluna.unique()
    for x in range(len(coluna)):
        if(coluna[x] == unicos[0]):
            coluna[x] = -1
        else:
            coluna[x] = 1
    return coluna

def criar_novas_colunas(coluna, df):
    for x in range(coluna.unique().size):
        df[str(coluna.name)+ "_" + coluna.unique()[x]] = tamanho
    return df

def acrescentar_nome_valores(coluna):
    for x in range(coluna.size):
        coluna[x] = coluna.name + "_" + coluna[x]

    return coluna   

In [76]:
df.school = converter_binario(df.school)
df.sex = converter_binario(df.sex)
df.address = converter_binario(df.address)
df.famsize = converter_binario(df.famsize)
df.Pstatus = converter_binario(df.Pstatus)
df.schoolsup = converter_binario(df.schoolsup)
df.famsup = converter_binario(df.famsup)
df.paid = converter_binario(df.paid)
df.activities = converter_binario(df.activities)
df.nursery = converter_binario(df.nursery)
df.higher = converter_binario(df.higher)
df.internet = converter_binario(df.internet)
df.romantic = converter_binario(df.romantic)

tamanho = []

for y in range(df.Mjob.size):
    tamanho.append(0)

g3 = df.pop("G3")

df = criar_novas_colunas(df.Mjob, df)

df = criar_novas_colunas(df.Fjob, df)

df = criar_novas_colunas(df.reason, df)

df = criar_novas_colunas(df.guardian, df)

df.Mjob = acrescentar_nome_valores(df.Mjob)

df.Fjob = acrescentar_nome_valores(df.Fjob)

df.reason = acrescentar_nome_valores(df.reason)

df.guardian = acrescentar_nome_valores(df.guardian)

df["G3"] = g3

for index, row in df.iterrows():
    df[row['Mjob']][index] = 1
    df[row['Fjob']][index] = 1
    df[row['reason']][index] = 1
    df[row['guardian']][index] = 1

#HTML(df.to_html(index=False))
df = df.drop(columns=['Mjob', 'Fjob', 'reason', 'guardian']) # comentar essa e a próxima linha para comparar mais fácilmente antes de retirar essas colunas, retirar o comentário anterior

#HTML(df.to_html(index=False))


In [77]:
for x in range(df.G3.size):
    if df.G3[x] < 14:
        df.G3[x] = "reprovado"
    else:
        df.G3[x] = "aprovado"


In [78]:
atributos = df[df.columns[:df.columns.size-1:]]

classe = np.array(df[df.columns[-1::]]).flatten()


In [79]:
def calcular_taxas_para_knn(k, qtd_taxas, porcent_test, atributos, classe):
    array_taxas = []

    for n in range(k):
        arrays = []
        for i in range(qtd_taxas):
            x_treino, x_teste, y_treino, y_teste = train_test_split(atributos, classe,test_size=porcent_test, random_state=i)
            classificador = KNeighborsClassifier(n_neighbors=n+1)
            classificador.fit(x_treino, y_treino)
            taxas = classificador.score(x_teste, y_teste)
            arrays.append(taxas)

        array_taxas.append(arrays)
        
    return array_taxas

array_taxas = calcular_taxas_para_knn(1, 100, 0.5, atributos, classe)

In [80]:
def calcular_intervalo_confianca(hash_medias, hash_diferencas_total):
    for i in range(len(hash_medias)):
        for x in range(len(hash_medias[i+1])):
            
            desvio_padrao = np.std(hash_diferencas_total[i+1][x])
            
            erro_padrao = desvio_padrao/np.sqrt(len(hash_diferencas_total[i+1][x]))

            multiplicador = abs(stats.distributions.norm.ppf(0.025))

            print(i+1, (hash_medias[i+1][x] - multiplicador*erro_padrao, hash_medias[i+1][x] + multiplicador*erro_padrao)
            )


In [81]:
def calcular_medias_separadas(array):

    hash_saida = {}
    array_aux = []
    for x in range(len(array)):  
        array_aux.append(np.mean(array[x]))
        hash_saida[x+1] = array_aux
        array_aux = []
    return hash_saida

medias_separadas = calcular_medias_separadas(array_taxas)

In [82]:
hash_taxas = {}
array_aux = []
for x in range(len(array_taxas)):
    array_aux.append(array_taxas[x])
    hash_taxas[x+1] = array_aux
    array_aux = []

calcular_intervalo_confianca(medias_separadas, hash_taxas)

1 (0.9091940388431689, 0.9153514157022852)
